In [3]:
import dashscope
from dashscope import Generation
dashscope.api_key="sk-4ba6df3298eb4ebab065b343770093b6"
response=Generation.call(
    model="qwen-turbo",
    prompt="帮我生成一个完整的大模型api示例文档"
)
print("AI回复:",response.output.text)


AI回复: 当然可以！以下是一个**完整的大模型 API 示例文档**，适用于一个通用的大型语言模型（如类似 GPT、BERT 或其他 NLP 模型）的 RESTful API 接口。该文档包括：

- **API 概述**
- **请求方法与路径**
- **请求参数**
- **响应格式**
- **错误码说明**
- **示例调用**
- **认证机制**

---

# 大模型 API 文档

## 1. API 概述

本 API 提供对大模型的访问接口，支持文本生成、问答、摘要、翻译等任务。通过 HTTP 请求与服务端进行交互。

### 1.1 版本信息
- **版本号**: `v1`
- **发布日期**: 2025-04-05

### 1.2 基础 URL
```
https://api.example.com/v1/models
```

---

## 2. 接口列表

| 路径 | 方法 | 描述 |
|------|------|------|
| `/generate` | POST | 生成文本 |
| `/chat` | POST | 对话式聊天 |
| `/summarize` | POST | 文本摘要 |
| `/translate` | POST | 文本翻译 |

---

## 3. 通用请求头

| 参数 | 必填 | 说明 |
|------|------|------|
| `Authorization` | 是 | Bearer Token，用于身份验证 |
| `Content-Type` | 是 | `application/json` |

---

## 4. 生成文本接口 (`/generate`)

### 4.1 请求方法
`POST`

### 4.2 请求体参数

| 参数 | 类型 | 必填 | 说明 |
|------|------|------|------|
| `prompt` | string | 是 | 输入提示文本 |
| `max_length` | integer | 否 | 最大输出长度，默认为 100 |
| `temperature` | float | 否 | 控制输出随机性，范围 [0, 1]，默认 0.7 |
| `top_p` | float | 否 | 采样概

In [6]:
import openai
client = openai.OpenAI(
    api_key=dashscope.api_key,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)
completion=client.chat.completions.create(
    model="qwen-turbo",
    messages=[
        {"role":"user","content":"who are you?"}
    ]
)
print(completion.model_dump_json())

{"id":"chatcmpl-c2e41dfe-2fc4-907b-9268-5b0fcb9ffec0","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"I am Qwen, a large-scale language model developed by Alibaba Group. I can answer questions, create text, write code, and more. How can I assist you today?","refusal":null,"role":"assistant","annotations":null,"audio":null,"function_call":null,"tool_calls":null}}],"created":1768355297,"model":"qwen-turbo","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":36,"prompt_tokens":16,"total_tokens":52,"completion_tokens_details":null,"prompt_tokens_details":{"audio_tokens":null,"cached_tokens":0}}}


In [9]:
from dashscope import Generation
messages = [
    {"role": "system", "content": "你是一名友好的AI助手。"},
    {"role": "user", "content": "你好！"},
    {"role": "assistant", "content": "你好，我是通义千问，很高兴认识你！"},
    {"role": "user", "content": "请用一句话解释人工智能是什么。"},
    {"role": "assistant", "content": "工智能是让机器模拟人类智能，以执行复杂任务的能力。"},
    {"role": "user", "content": "帮我写一首诗"},
]
response = Generation.call(
    model="qwen-turbo",
    messages=messages
)
print("AI回复：", response.output.text)

AI回复： 春风拂面花初绽，  
柳絮轻飞入画栏。  
鸟语声中藏古意，  
心随云去自悠然。


In [13]:
import time
from dashscope import Generation

def chat_with_ai(prompt,retries=3):
    for i in range(retries):
        try:
            response = Generation.call(
                model="qwen-turbo",
                prompt=prompt,
                temperature=0.8
            )
            return response.output.text
        except Exception as e:
            print(f"第{i+1}次尝试失败:{e}")
            time.sleep(2)
    return "AI暂时无法响应，请稍后再试。"
# test
print(chat_with_ai("请写一首关于拉萨的诗"))

**拉萨**

晨光洒在布达拉宫的红墙，  
风穿过八廓街的经幡，  
转经的人影，如流水般绵延，  
心在诵经声中慢慢沉淀。

酥油灯点亮了古老的信仰，  
玛尼堆上刻着岁月的低语，  
雪山之巅，云朵轻轻飘过，  
仿佛天神也在凝望人间。

大昭寺前，香火缭绕，  
朝圣者跪地，虔诚如初，  
每一滴汗水，都是对灵魂的叩问，  
每一步脚印，都通向心灵的归途。

拉萨河静静流淌，  
带走了尘世的喧嚣，  
只留下高原的宁静与辽阔，  
和那颗永不褪色的信仰。


In [14]:
import dashscope
import gradio as gr
from dashscope import Generation
dashscope.api_key = "sk-4ba6df3298eb4ebab065b343770093b6"

def chat_with_ai(message,history):
    try:
        conversation = "\n".join([f"用户: {h[0]}\nAI: {h[1]}" for h in history if h])
        prompt = conversation + "\n用户:" + message
        res = Generation.call(model="qwen-turbo",prompt=prompt)
        answer = res.output.text
        return answer
    except Exception as e:
        return f"出错:{e}"
# 使用ChatInterface实现聊天模式
chatbot = gr.ChatInterface(
        fn=chat_with_ai,
        title="💬 通义千问 · 智能聊天助手",
        description="和AI聊聊天吧！它能写诗、答疑、讲故事，还能帮你写代码哦～"
        )
chatbot.launch()




D:\miniconda\envs\ai251218\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [15]:
import dashscope
import gradio as gr
from dashscope import Generation
dashscope.api_key = "sk-4ba6df3298eb4ebab065b343770093b6"

def chat_with_ai(message,history):
    try:
        messages = [
            {"role": "system", "content": "你是一个有帮助的中文助手。"}
        ]

        for item in history:
            # 情况 1：老版 history -> [user, bot]
            if isinstance(item, (list, tuple)) and len(item) == 2:
                user_msg, bot_msg = item
                messages.append({"role": "user", "content": str(user_msg)})
                messages.append({"role": "assistant", "content": str(bot_msg)})

            # 情况 2：新版 history -> {"role": "...", "content": ...}
            elif isinstance(item, dict) and "role" in item and "content" in item:
                role = item.get("role", "user")
                content = item.get("content", "")

                # content 可能是 list（多模态），这里简单拼成字符串
                if isinstance(content, list):
                    content = " ".join(
                        c.get("text", "") if isinstance(c, dict) else str(c)
                        for c in content
                    )
                elif not isinstance(content, str):
                    content = str(content)

                messages.append({"role": role, "content": content})

            # 兜底：任何奇怪结构，都当作用户一句话
            else:
                messages.append({"role": "user", "content": str(item)})

        # 当前这一轮用户输入
        messages.append({"role": "user", "content": message})

        res = Generation.call(
            model="qwen-turbo",
            messages=messages,
            result_format="message",
        )
        return res.output.choices[0]["message"]["content"]
    except Exception as e:
        return f"出错：{repr(e)}"

# 使用ChatInterface实现聊天模式
chatbot = gr.ChatInterface(
        fn=chat_with_ai,
        title="💬 通义千问 · 智能聊天助手",
        description="和AI聊聊天吧！它能写诗、答疑、讲故事，还能帮你写代码哦～"
        )
chatbot.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
